# Libraries

In [7]:
import argparse
import logging
import numpy as np
import pandas as pd
import random
import sys
import operator
from math import sqrt

from sklearn.model_selection import train_test_split
from sklearn.utils.validation import check_random_state
from sklearn.metrics import mean_absolute_error, mean_squared_error
import graphviz
from collections import OrderedDict
from sympy import simplify

# For Support Vector Regression
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# For Random Forest Regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold


# For GPLearn
from gplearn.genetic import SymbolicRegressor
from gplearn.functions import make_function

# For FFX estimator
import ffx

# For QLattice
import feyn
# ql = feyn.connect_qlattice()

# For DEAP
# from tpot import TPOTRegressor

from pyshgp.push.instruction_set import InstructionSet
from pyshgp.gp.estimators import PushEstimator
from pyshgp.gp.genome import GeneSpawner

# Checks time
from time import process_time

# Datasets setup

In [2]:
# Read compiled dataset with number of features and observations on the 50 assigned datasets
df = pd.read_csv("../Datasets/All_Datasets_Stats.csv")
df.head()


,File,Number of observations,Number of features
0,body_fat.csv,252,14
1,Hill_Valley_with_noise.csv,1212,100
2,allbp.csv,3772,29
3,analcatdata_cyyoung9302.csv,92,10
4,229_pwLinear.csv,200,10


# Basic implementation of regression systems

In [9]:
# GPLearn (Genetic Programming)
def GP(POP, GEN, CXPB, PC):
    est_gp = SymbolicRegressor(population_size=POP,
                           generations=GEN,p_crossover=CXPB, p_subtree_mutation=0,
                           p_hoist_mutation=0, p_point_mutation=0,
                           max_samples=0.9, verbose=1,
                           parsimony_coefficient=PC, random_state=0)
    
    return est_gp 

# Random Forest Regressor
def RR(NUM, DEPTH, JOB, VER):
    est_rr = RandomForestRegressor(n_estimators=NUM, max_depth=DEPTH, n_jobs= JOB, verbose=VER, random_state=0)
    return est_rr 

# Support Vector Regression
def SVR(C, EPI, CS , ITER):
    est_svr = make_pipeline(StandardScaler(), SVR(C=C, epsilon=EPI, cache_size = CS, max_iter = ITER))
    return est_svr 

df1 = pd.read_csv("../Datasets/Hill_Valley_with_noise.csv")
size = len(df1.columns)-1
X = df1.drop(['target'], axis=1)
y = df1['target']

# dividing X and y into training and testing units
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=1)
gp = GP(1021 , 40, 0.54 , 0.73)
gp.fit(X_train, y_train)
y_test_pred = gp.predict(X_test)
mae = mean_absolute_error(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    26.15      7.12135e+55       63         0.486899         0.602664     21.66s
   1     4.52      1.58945e+38        3         0.494102         0.576471     14.11s
   2     2.93          889.644        5         0.482453          0.65312     14.67s
   3     2.41          774.002        3         0.486239         0.647059     13.51s
   4     1.01          8.92878        3          0.48886         0.623529     12.18s
   5     1.00           1.4802        1          1.46245          1.64159     11.78s
   6     1.00          1.48045        1          1.46114          1.65335     11.57s
   7     1.00          1.48062        1          1.46376          1.62982     11.00s
   8     1.00          1.48047        1          1.46245          1.64159  

In [11]:
df = pd.read_csv("../Datasets/Hill_Valley_with_noise.csv")
import math
        
size = len(df.columns)-1
X = df.drop(['target'], axis=1)
y = df['target']

# dividing X and y into training and testing units
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = True, random_state=1)
n_train = math.ceil(X.shape[0] * 0.7)

X_train = X.iloc[:n_train, :]
y_train = y.iloc[:n_train]

X_test = X.iloc[n_train:, :]
y_test = y.iloc[n_train:]

scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

INSTANCES = "i4"




Use 3th-cv split


In [4]:
# Checks time
from time import process_time

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# Symbolic regression parameters
pop = [int(x) for x in np.linspace(start = 1000, stop = 1500)]
generation = [int(x) for x in np.linspace(start = 10, stop = 50)]
crossover = [float(x) for x in np.linspace(0.01, 1)]
parsimony_coefficient = [float(x) for x in np.linspace(start = 0.01, stop = 1)]

# Random forest parameters
n_estimators = [int(x) for x in np.linspace(start = 1, stop = 500)]
n_jobs = [int(x) for x in np.linspace(start = 1, stop = 20)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 20)]
verbose = [int(x) for x in np.linspace(start = 0, stop = 20)]

# SVR parameters
c = [int(x) for x in np.linspace(start = 1, stop = 20, num = 20)]
epsilon = [float(x) for x in np.linspace(start = 0.1, stop = 20)]
cache_size = [int(x) for x in np.linspace(1, 1000)]
max_iter = [int(x) for x in np.linspace(start = 1, stop = 100)]

datarr = pd.DataFrame()

def calculateTime(regressor):
#     df1 = pd.read_csv("../Datasets/"+ str(df.File[i]))
    df1 = pd.read_csv("../Datasets/Hill_Valley_with_noise.csv")
    size = len(df1.columns)-1
    X = df1.drop(['target'], axis=1)
    y = df1['target']

    # dividing X and y into training and testing units
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state=1)

    # Create the random grid
    random_grid = {}

    if regressor == "GPLearn":
        rf = SymbolicRegressor(p_subtree_mutation=0,
                       p_hoist_mutation=0, p_point_mutation=0,
                       max_samples=0.9, verbose=1,random_state=0)
        random_grid = {'population_size': pop,
                   'generations': generation,
               'p_crossover': crossover,
               'parsimony_coefficient': parsimony_coefficient}
        rf_random = GridSearchCV(estimator = rf, param_grid = random_grid)
    elif regressor == "RR":
        rf = RandomForestRegressor()
        random_grid = {'n_estimators': n_estimators,
                   'n_jobs': n_jobs,
                   'max_depth': max_depth,
                   'verbose': verbose}
        rf_random = GridSearchCV(estimator = rf, param_grid = random_grid)
    else:
        random_grid = {'rbf_svm__C': c,
                   'rbf_svm__epsilon': epsilon,
                   'rbf_svm__cache_size': cache_size,
                   'rbf_svm__max_iter': max_iter}
        steps = [('scaler', StandardScaler()), ('rbf_svm', SVR())]
        pipeline = Pipeline(steps)
        # do search
        rf_random = GridSearchCV(pipeline, param_grid=random_grid)
    # Fit the random search model
    rf_random.fit(X_train, y_train)

    return rf_random.best_params_, rf_random

In [5]:
# start = process_time()
# bp, rf_random = calculateTime("GP")


# y_test_pred = rf_random.predict(X_test)
# # Mean absolute error
# mae = mean_absolute_error(y_test, y_test_pred)
# # Mean squared error to check performance
# mse = mean_squared_error(y_test, y_test_pred)
# # Root mean squared error to check accuracy
# rmse = sqrt(mse)

# stop = process_time()
# tt = stop-start
# models = {'GPLearn Symbolic Regression': ,
#           'Random Forest Regression': RR(),
#           'Support Vector Regression': SVR()
#          }

import pandas as pd

df1 = pd.read_csv("../Datasets/Hill_Valley_with_noise.csv")
size = len(df.columns)-1
X = df1.drop(['target'], axis=1)
y = df1['target']

from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = True, random_state=1)

tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))


Generation 1 - Current best internal CV score: -0.16635826639702408

Generation 2 - Current best internal CV score: -0.05710647673039142

Generation 3 - Current best internal CV score: -0.05710647673039142

Generation 4 - Current best internal CV score: -0.05710647673039142

Generation 5 - Current best internal CV score: -0.05710647673039142

Best pipeline: DecisionTreeRegressor(Normalizer(input_matrix, norm=max), max_depth=3, min_samples_leaf=1, min_samples_split=8)
-0.05972384415698741


In [8]:
print(tt)
print(mae)
print(mse)
print(rmse)
epsilon

NameError: name 'tt' is not defined

In [ ]:
tt, mae, mse, rmse = calculateTime("GPLearn")
print(tt)
print(mae)
print(mse)
print(rmse)

In [11]:
tt, mae, mse, rmse = calculateTime("SVR")
print(tt)
print(mae)
print(mse)
print(rmse)

TypeError: SVR() missing 3 required positional arguments: 'EPI', 'CS', and 'ITER'

In [45]:
# SVR approach
best_SVR_non = []
start = process_time()

cur_rmse = 10000000

for x in range(1,20):
    for y in range(1,20):
            total = SVR(x,y)
            total.fit(X_train, y_train)
            y_test_pred = total.predict(X_test)
            mae = mean_absolute_error(y_test, y_test_pred)
            mse = mean_squared_error(y_test, y_test_pred)
            rmse = sqrt(mse)
            if rmse<cur_rmse:
                best_num = x
                best_depth = y 
                best_SVR_non = [x,y]
stop = process_time()
print('Time: ', stop - start)  

print(best_SVR_non)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass kernel=1, degree=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


NameError: name 'X_train' is not defined

In [ ]:
# for model_name, model_instance in models.items():
#     print('Training model {}'.format(model_name))
#     model_instance.fit(X_train, y_train)

In [452]:
# Evaluation
for model_name, model_instance in models.items():
    y_test_pred = model_instance.predict(X_test)
    mae = mean_absolute_error(y_test, y_test_pred)
    mse = mean_squared_error(y_test, y_test_pred)
    rmse = sqrt(mse)
    
    print('Model {}: \n mae: {} \n mse: {} \nrmse: {} \n'.format(model_name, mae, mse,rmse))

Model GPLearn Symbolic Regression: 
 mae: 0.6365963201245488 
 mse: 0.6062201564526084 
rmse: 0.7786014105128557 

Model Random Forest: 
 mae: 0.40887500000000004 
 mse: 0.31670725000000005 
rmse: 0.5627674919538264 

Model Support Vector Regression: 
 mae: 0.4735990938254245 
 mse: 0.38761639573435075 
rmse: 0.6225884641834851 



In [431]:
# Test if all symbolic regression systems works
# simplify(models[0].sympify())

## Optional: To create tree diagrams

In [432]:
# Print fittest solution
print(models['GPLearn Symbolic Regression']._program)

mul(sub(X3, X0), 0.140)


In [434]:
# Export to a graph instance
graph = models['GPLearn Symbolic Regression']._program.export_graphviz()  
graph_str = str(graph)
program_str = str(models['GPLearn Symbolic Regression']._program)

# Replace X{} with actual features names
mapping_dict = {'X{}'.format(i): X.columns[i] for i in reversed(range(X.shape[1]))}
for old_value, new_value in mapping_dict.items():
    graph_str = graph_str.replace(old_value, str(new_value))
    program_str = program_str.replace(old_value, str(new_value))

    
# Save localy
src = graphviz.Source(graph_str)
src.render('result.gv', view=True)

'result.gv.pdf'

In [322]:
program_str

'mul(sub(oz4, oz1), 0.140)'

NameError: name 'rf' is not defined